In [2]:
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

--2023-04-26 21:20:48--  https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 0.0.0.0, ::
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|0.0.0.0|:443... failed: Connection refused.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|::|:443... failed: Connection refused.


In [3]:
import torch
from unixcoder import UniXcoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UniXcoder("microsoft/unixcoder-base")
model.to(device)

UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [4]:
import pandas as pd
import numpy as np
import  re
from torch import per_channel_affine
from tqdm import tqdm
import time, random
import torch

import json
import cv2
import numpy as np
from tqdm import tqdm
import math
from math import log
from torch.nn.utils.rnn import pad_sequence
import sys
import time
import os
from collections import defaultdict, Counter
from multiprocessing import Pool
from functools import partial
from itertools import chain

获取数据文件（先找个小一点的jsonl文件放到服务器上，选择CSN里ruby语言的test文件比较小）

In [5]:
from transformers import AutoTokenizer, AutoModel

# Load pre-trained unixcoder tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('microsoft/unixcoder-base')
model = AutoModel.from_pretrained('microsoft/unixcoder-base')


In [6]:
#input:code snippet
import jsonlines

test_list=[]
test_file="/home/huangyx/emscore/emscore/dataset/ruby_test_0.jsonl"
with open(test_file,mode='r') as f:
    for item in jsonlines.Reader(f):
        test_list.append(item)
print(len(test_list))
    

2279


In [7]:
#gain sample
import json

sample_list=random.sample(test_list,10)
print(sample_list[0]['code'])
print("----------------------------------------")
print(sample_list[0]['docstring'])


def build_sale_data(sale_item)
        "".b.tap() do |data|
          price_units = (sale_item.price * 100).to_i # !FIXME
          price_bytes = "".b
          4.times{ |shift| price_bytes.insert 0, ((price_units >> shift*8) & 0xff).chr }
          data << price_bytes
          qty_units = ((sale_item.qty || 1) * 1000).to_i # !FIXME
          qty_bytes = "".b
          4.times{ |shift| qty_bytes.insert 0, ((qty_units >> shift*8) & 0xff).chr }
          data << qty_bytes
          data << "\x00".b #number len FIXME
          data << "\xAA\xAA\xAA\xAA\xAA\xAA".b #number FIXME
          text = sale_item.text1.truncate(20)
          data << text.length.chr
          data << text.ljust(20, " ").b
          data << (sale_item.tax_group || 2).chr
        end
      end
----------------------------------------
def build_sale_data(price, text1 = "", text2 = nil, tax_group = 2, qty = 1, percent = nil, neto = nil, number = nil)


In [10]:
#得到code和candidate的embedding向量,output_code/ouput_candidate就包含了每一个token/word向量
#code_embeddings  candidate_embeddings就是分别对应整个向量表示，这里取的是mean操作获得



# Encode input code and obtain word embeddings
input_code = sample_list[0]['code']
encoded_code = tokenizer.encode_plus(input_code, return_tensors='pt')
output_code = model(**encoded_code)[0]
code_embeddings = output_code.mean(dim=1)

# Encode input candidate and obtain word embeddings
input_candidate = sample_list[0]['docstring']
encoded_candidate = tokenizer.encode_plus(input_candidate, return_tensors='pt')
output_candidate = model(**encoded_candidate)[0]
candidate_embeddings = output_candidate.mean(dim=1)


In [8]:
print(test_list[1]['code'] )
print("-------------------")
print(test_list[1]['docstring'])

def print_summary(status)
      status_string = status.to_s.humanize.upcase
      if status == :success
        heading("Result: ", status_string, :green)
        level = :info
      elsif status == :timed_out
        heading("Result: ", status_string, :yellow)
        level = :fatal
      else
        heading("Result: ", status_string, :red)
        level = :fatal
      end

      if (actions_sentence = summary.actions_sentence.presence)
        public_send(level, actions_sentence)
        blank_line(level)
      end

      summary.paragraphs.each do |para|
        msg_lines = para.split("\n")
        msg_lines.each { |line| public_send(level, line) }
        blank_line(level) unless para == summary.paragraphs.last
      end
    end
-------------------
Outputs the deferred summary information saved via @logger.summary.add_action and @logger.summary.add_paragraph


In [ ]:
print(output_code.shape)
print(output_candidate.shape)

In [9]:
#这里的a参数指的是每一个candidate文本句子

def process(a, tokenizer=None):
    if tokenizer is not None:
        a1 = tokenizer.encode(a, add_special_tokens=True, return_tensors="pt").tolist()[0]
    return set(a1)

In [10]:
#算词频，一共N个句子，统计单词在这些句子里出现几次，出现在一个句子里算一次，输入一个corpus里所有句子，输出所有单词对应的词频

def get_idf_dict(arr, tokenizer, nthreads=4):
    """
    Returns mapping from word piece index to its inverse document frequency.
    Args:
        - :param: `arr` (list of str) : sentences to process.
        - :param: `tokenizer` : a BERT tokenizer corresponds to `model`.
        - :param: `nthreads` (int) : number of CPU threads to use
    """
    idf_count = Counter()
    num_docs = len(arr)

    process_partial = partial(process, tokenizer=tokenizer)

    with Pool(nthreads) as p:
        idf_count.update(chain.from_iterable(p.map(process_partial, arr)))

    idf_dict = defaultdict(lambda: log((num_docs + 1) / (1)))
    idf_dict.update({idx: log((num_docs + 1) / (c + 1)) for (idx, c) in idf_count.items()})
    return idf_dict

In [11]:
candidate_list=[]
for item in test_list:
    candidate_list.append(item['docstring'])



In [ ]:
#计算得到每个token对应的idf结果

idf_dict=get_idf_dict(candidate_list,tokenizer)
print(idf_dict)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/unixcoder-base')
model = AutoModel.from_pretrained('microsoft/unixcoder-base')

In [15]:
def encode_text(vid_caps, model, tokenizer, idf_dict, device):
    with torch.no_grad():
        text_input = tokenizer.encode_plus(vid_caps, return_tensors='pt',return_attention_mask=True)

        text_features = model(**text_input)[0]
    #mask= text_input['attention_mask']

    text_features /= text_features.norm(dim=-1, keepdim=True)

    # For special tokens, use [SOS] and [EOS]
    txt_len = text_input['input_ids'].argmax(dim=-1)
    mask = torch.zeros_like(text_input['input_ids'])
    for i in range(len(mask)):
        mask[i][0:txt_len[i]+1] = 1

    idf_weights = torch.tensor([[idf_dict[int(i)] for i in a] for a in text_input['input_ids'].cpu()])

    return text_features, mask, idf_weights


In [18]:
#vid_caps='Outputs the deferred summary information saved via @logger.summary.add_action and @logger.summary.add_paragraph'

vid_caps='A person is frying a pan of eggs that are colored pink and green.'
device="cuda:0"
text_features,mask,idf_weights=encode_text(vid_caps,model,tokenizer,idf_dict,device)
print("文本处理之后的一些信息：")
print(type(text_features))
print(text_features)
print(text_features.shape)

print("关于mask的一些信息")
print(type(mask))
print(mask)
print(mask.shape)

print("关于idf权重的一些信息：")
print(type(idf_weights))
print(idf_weights)
print(idf_weights.shape)

文本处理之后的一些信息：
<class 'torch.Tensor'>
tensor([[[-0.0259, -0.0197, -0.0466,  ..., -0.0349,  0.0600,  0.0107],
         [-0.0259, -0.0197, -0.0466,  ..., -0.0349,  0.0600,  0.0107],
         [-0.0259, -0.0197, -0.0466,  ..., -0.0349,  0.0600,  0.0107],
         ...,
         [-0.0604,  0.0047, -0.0462,  ...,  0.0207, -0.0056,  0.0171],
         [ 0.0185,  0.0171, -0.0471,  ..., -0.0024,  0.0075, -0.0214],
         [-0.0312, -0.0250, -0.0111,  ...,  0.0002, -0.0119,  0.0200]]])
torch.Size([1, 22, 768])
关于mask的一些信息
<class 'torch.Tensor'>
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
torch.Size([1, 22])
关于idf权重的一些信息：
<class 'torch.Tensor'>
tensor([[0.0000, 4.5131, 6.3456, 1.4515, 4.8416, 6.3456, 2.8341, 0.8546, 7.7319,
         1.1145, 6.3456, 5.6525, 1.9543, 2.2939, 5.7860, 7.0388, 5.3340, 7.7319,
         1.3969, 6.6333, 0.5534, 0.0000]])
torch.Size([1, 22])


对文本处理的部分处理完了，接下来应该把原来对video进行处理的部分改成对code进行处理，但本质上对code处理的过程应该跟text很类似，只是得到code中每个token的向量后，如何将其组合得到不同粒度的向量表示